In [5]:
import pandas as pd
from textblob import TextBlob
import datetime
import time
import string
from string import punctuation

#df2 = pd.read_csv('df_mega_eth.csv')
#df3 = pd.read_csv('df_mega_binance.csv')
df = pd.read_csv('__SELF_CONTAINED/1)DATA/reddit_posts/df_mega_crypmkts_117.csv')
df = df.sort_values('timestamp')
#df5 = pd.read_csv('df_mega_bitcoinnews.csv')
#df6 = pd.read_csv('df_mega_block.csv')

df1 = df[0:103430]
df1 = df[103430:206860]
df2 = df[206860:310290]
#df3 = df[310290:413720]
#df4 = df[413720:517150]
#df5 = df[517150:620580]
#df6 = df[620580:724010]
#df7 = df[724010:827440]
#df8 = df[827440:930870]
#df9 = df[930870:1034300]
#df10 = df[1034300:1137730]
#df11 = df[1137730:1241160]
#df12 = df[1241160:1344590]
#df13 = df[1344590:1448020]
#df14 = df[1448020:1551450]

#print(len(df5))
#print(len(df8))

len(df)
len(df1)

20976

first we strip of punctuation

In [6]:
def strip_punctuation(s):
    return ''.join(c for c in s if c not in ascii(string.punctuation))

In [7]:
print(string.punctuation) #check if apostrophe unicode

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [8]:
#convert to string first
df1['text'] = [str(item) for item in df1['text']]

df1['text'] = [strip_punctuation(item) for item in df1['text']]
#     for some reason does not get rid of all apostrophes upon inspecting
#     the dataframe so we handle that later

#  now make it text blob objects
textblob_objects = [TextBlob(item) for item in df1['text']]
df1['text'] = textblob_objects

/Users/Alexandra/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/Alexandra/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/Alexandra/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

In [9]:
df1['timestamp'] = df1['timestamp'].dropna()
df1 = df1[df1['timestamp'].isnull() == False] #i think does same thing as one above

/Users/Alexandra/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [10]:
int_time1 = df1['timestamp'].apply(int) 

#^^^^sometimes need to toggle comment, above line if rerunning cell for second time will give error

df1['timestamp'] = [datetime.datetime.fromtimestamp(item) for item in int_time1]
print(len(df1))

20976


In [4]:

backs_list = ['back','backs','backed','backing','support','supports',
              'supporting', 'endorse', 'endorsing','endorsed']


bans_list = ['bans','banned','banning','ban','opposes','oppose','opposed','anti','against']

cryptolist = ['bitcoin','ethereum','ripple','btc','eth',
              'xrp','crypto','cryptocurrency','blockchain']

The functions findPosGrams and findNegGrams take in a textBlob object (which is now what every element under df1['text'] is.. and then check for the occurence of the possible gram: 'NNP' 'backs' 'bitcoin' .... or similar styles of 3-grams. Note that the first element is the Part Of Speech embedding of the textblob tagged object.

these functions have been tested and work on dataframes about 5000 rows long but always end up breaking or taking too long for things larger, and a try except was put into place because i kept getting an index out of range when I shouldn't have been (investigated all cases because the " 's " is considered its own seperate word when breaking the blob into its tagged parts)

In [23]:

def findPosGrams(blob2):
    num = len(blob2.words) - 2
    i = 0
    count = 0
    try:
        for a in range(0,len(backs_list)):
            for b in range(0,len(cryptolist)):
                for j in range(0,num):
                    if blob2.tags[j][i+1] == 'NN' and blob2.tags[j+1][i].lower() == backs_list[a] and blob2.tags[j+2][i].lower() == cryptolist[b]:
                        count += 1
                        print('hi')
                    if blob2.tags[j][i+1] == 'NNP' and blob2.tags[j+1][i].lower() == backs_list[a] and blob2.tags[j+2][i].lower() == cryptolist[b]:
                        count += 1
                        print('ho')
                    if blob2.tags[j][i+1] == 'NN' and blob2.tags[j+1][i].lower() == backs_list[a] and blob2.tags[j+2][i].lower() == "'s" and blob2.tags[j+3][i].lower() == cryptolist[b]:
                        count += 1
                        print('hi8')
                    if blob2.tags[j][i+1] == 'NNP' and blob2.tags[j+1][i].lower() == backs_list[a] and blob2.tags[j+2][i].lower() == "'s" and blob2.tags[j+3][i].lower() == cryptolist[b]:
                        count += 1
                        print('hi9')
    except IndexError:
        count = 0
        
    return count

In [24]:

from textblob import TextBlob
blob1 = TextBlob("but really JPMorgan Back's ripple China Back's Bitcoin hello")
print(blob1.tags)
print(blob1.words)


[('but', 'CC'), ('really', 'RB'), ('JPMorgan', 'NNP'), ('Back', 'NNP'), ("'s", 'POS'), ('ripple', 'NN'), ('China', 'NNP'), ('Back', 'NNP'), ("'s", 'POS'), ('Bitcoin', 'NNP'), ('hello', 'NN')]
['but', 'really', 'JPMorgan', 'Back', "'s", 'ripple', 'China', 'Back', "'s", 'Bitcoin', 'hello']


In [25]:
findPosGrams(blob1)

hi9
hi9


2

In [28]:
def findNegGrams(blob2):
    num = len(blob2.words) - 2
    i = 0
    count = 0
    try:
        for a in range(0,len(bans_list)):
            for b in range(0,len(cryptolist)):
                for j in range(0,num):
                    if blob2.tags[j][i+1] == 'NN' and blob2.tags[j+1][i].lower() == bans_list[a] and blob2.tags[j+2][i].lower() == cryptolist[b]:
                        count += 1
                        print('hi')
                    if blob2.tags[j][i+1] == 'NNP' and blob2.tags[j+1][i].lower() == bans_list[a] and blob2.tags[j+2][i].lower() == cryptolist[b]:
                        count += 1
                        print('ho')
                    if blob2.tags[j][i+1] == 'NN' and blob2.tags[j+1][i].lower() == bans_list[a] and blob2.tags[j+2][i].lower() == "'s" and blob2.tags[j+3][i].lower() == cryptolist[b]:
                        count += 1
                        print('hi8')
                    if blob2.tags[j][i+1] == 'NNP' and blob2.tags[j+1][i].lower() == bans_list[a] and blob2.tags[j+2][i].lower() == "'s" and blob2.tags[j+3][i].lower() == cryptolist[b]:
                        count += 1
                        print('hi9')
    except IndexError:
        count = 0
        
    return count

In [29]:
blob_neg = TextBlob("but really JPMorgan ban's ripple China ban's Bitcoin hello")
findNegGrams(blob_neg)

hi9
hi9


2

###### ok see if this works, ok it doesn't

In [15]:
#df1['neg_ngram_count'] = df1['text'].apply(findNegGrams)

In [5]:
df1 = df1.sort_values('timestamp')

In [4]:
df1['day'] = [item.strftime('%Y-%m-%d') for item in df1['timestamp']]

In [ ]:
df1_tstp_bigr = df1.filter(items=['timestamp', 'pos_gram_count'])

In [ ]:
df_negcount1 = df1.filter(items=['day', 'neg_gram_count'])
df_poscount1 = df1.filter(items=['day', 'pos_gram_count'])

In [ ]:
df_negperday1 = df_negcount1.groupby('day').sum()
df_posperday1 = df_poscount1.groupby('day').sum()

In [ ]:
df_posperday1.to_csv('df_posperday_bitcoin_df1_.csv')
df_negperday1.to_csv('df_negperday_bitcoin_df1_.csv')

In [ ]:
import matplotlib.pylab as plt
%matplotlib inline
import seaborn; seaborn.set()

#plt.figure(figsize=(12,6))
fig, ax = plt.subplots(figsize = (12,6))

plt.plot(df_negperday1)
plt.plot(df_posperday1) #,marker = 'o', linestyle = 'None')

#beautiful, text itself is tick labels. the 3 is step
step = 3
ax.xaxis.set_ticklabels(df_negperday1.index.values[::step]) #.index as opposed to ['day']
#tick labels are the text that appear, need to be same size 
#so they know at every spot, what do i place?
ax.xaxis.set_ticks(df_negperday1.index.values[::step]) #where i want labels to go
_ = plt.xticks(rotation = 70)


#probably an ovelapping histogram
#sparse ticks, and add plt legend